In [ ]:
!pip install pywapor==2.3.1 --quiet

In [1]:
import pywapor

project_folder = r"/Users/hmcoerver/On My Mac/pyWAPOR/example_data"
latlim = [28.9, 29.7]
lonlim = [30.2, 31.2]
startdate = "2021-06-01"
enddate = "2021-08-01"

In [ ]:
composite_length = "DEKAD"

epochs_info = pywapor.pre_et_look.create_dates(startdate, enddate, composite_length)

for number, start, end in zip(*epochs_info):
    print(f"Composite {number + 1} starts at {start} and ends at {end}.\n")

In [ ]:
import os

raw_folder = os.path.join(project_folder, "RAW")

dl_args = (raw_folder, latlim, lonlim, startdate, enddate)

raw_ndvi_files = list()
raw_ndvi_files.append(pywapor.collect.MOD13.NDVI(*dl_args))
raw_ndvi_files.append(pywapor.collect.MYD13.NDVI(*dl_args))

In [ ]:
cmeta = {
    "composite_type": "mean",
    "temporal_interp": False,
    "spatial_interp": "nearest",
    "var_name": "ndvi",
    "var_unit": "-",
}

ds = pywapor.general.compositer.main(cmeta, raw_ndvi_files, epochs_info, lean_output = False)

In [ ]:
ds

In [ ]:
diagnostics = { # label          # lat      # lon
                "agriculture":	(29.32301,	30.77599),
              }

pywapor.post_et_look.plot_composite(ds, diagnostics)

In [ ]:

for number, start, end in zip(*epochs_info):
    print(f"Composite {number + 1} starts at {start} and ends at {end}.\n")

In [ ]:
epochs_info = pywapor.pre_et_look.create_dates(startdate, enddate, 6)
ds = pywapor.general.compositer.main(cmeta, raw_ndvi_files, epochs_info, lean_output = False)
pywapor.post_et_look.plot_composite(ds, diagnostics)

In [ ]:
cmeta = {
    "composite_type": "mean",
    "temporal_interp": "linear",
    "spatial_interp": "nearest",
    "var_name": "ndvi",
    "var_unit": "-",
}

ds = pywapor.general.compositer.main(cmeta, raw_ndvi_files, epochs_info, lean_output = False)
pywapor.post_et_look.plot_composite(ds, diagnostics)

In [ ]:
cmeta = {
    "composite_type": "max",
    "temporal_interp": "linear",
    "spatial_interp": "nearest",
    "var_name": "ndvi",
    "var_unit": "-",
}

ds = pywapor.general.compositer.main(cmeta, raw_ndvi_files, epochs_info, lean_output = False)
pywapor.post_et_look.plot_composite(ds, diagnostics)